In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
CSV_FILE = 'data/hob_to_33s_2022-11-20.csv'
df = pd.read_csv(CSV_FILE)
date_fields = ['fetch_time', 'projected_arrival', 'last_updated', 'door_close', 'scheduled_departure']
for field in date_fields:
    df[field] = pd.to_datetime(df[field], format='ISO8601')

In [ ]:
# all values are the same
door_close = df.door_close.mode()[0]
scheduled_departure = df.scheduled_departure.mode()[0]

df['predicted_lateness_min'] = (df.projected_arrival - df.scheduled_departure).dt.total_seconds() / 60
df['actual_lateness_min'] = (df.door_close - df.scheduled_departure).dt.total_seconds() / 60

In [ ]:
print('Final error (minutes) in predicted vs actual lateness:', df.iloc[-1].predicted_lateness_min - df.iloc[-1].actual_lateness_min)

In [ ]:
def make_graph(df, lateness=False):
    plt.figure(figsize=(10, 6))
    df_official = df[df.api == 'OFFICIAL']
    df_mrazza = df[df.api == 'MRAZZA']
    
    # leave time
    walk_time_min = 10
    ts = [df.iloc[0].fetch_time, df.iloc[120].fetch_time]
    ys = [(t - scheduled_departure).total_seconds() / 60 + walk_time_min for t in ts]
    
    ys = [-1, df.iloc[-1].actual_lateness_min + 1]
    ts = [pd.Timedelta(seconds=(y - walk_time_min) * 60) + scheduled_departure for y in ys]

    plots = [
        {
            'x': df_official['fetch_time'],
            'y': 0,
            'color': 'yellow',
            'label': 'schedule',
        },
        {
            'x': df_official['fetch_time'],
            'y': df_official['predicted_lateness_min'],
            'color': 'blue',
            'label': '"official" API',
        },
        {
            'x': df_mrazza['fetch_time'],
            'y': df_mrazza['predicted_lateness_min'],
            'color': 'green',
            'label': 'mrazza API',
        },
        {
            'x': df_official['fetch_time'],
            'y': df_official['actual_lateness_min'],
            'color': 'orange',
            'label': 'actual',
        },
        {
            'x': pd.Series(ts),
            'y': pd.Series(ys),
            'color': 'purple',
        },
    ]
    
    for plot in plots:
        if not lateness:
            plot['y'] += (scheduled_departure - plot['x']).dt.total_seconds() / 60

        sns.lineplot(**plot)

    # actual departure (door close) time
    plt.axvline(door_close, color='red', linestyle='--')

    plt.legend(loc='lower left')
    

make_graph(df, lateness=True)